In [2]:
%pip install openai python-dotenv 


  Using cached openai-2.6.1-py3-none-any.whl.metadata (29 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.11.1-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.4-cp311-cp311-win_amd64.whl.metadata (7.4 kB)
  Using cached

In [3]:
# 3) Cargar la clave y crear el cliente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


In [4]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías y algoritmos que permiten personalizar y optimizar el aprendizaje, adaptándose a las necesidades y ritmos de cada estudiante. Estas herramientas pueden incluir sistemas de tutoría inteligente, análisis de datos para mejorar la enseñanza y recursos educativos interactivos que fomentan un aprendizaje más efectivo y accesible.


In [5]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.3,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida conocida, lo que permite al modelo aprender a predecir resultados a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida conocida, lo que permite al modelo aprender a predecir resultados a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión.'}


## 7) Ejercicios propuestos
1. Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
2. Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
3. Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.
4. Limita la longitud con `max_tokens` y observa si corta.


In [ ]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.1,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura jerárquica para tomar decisiones basadas en características de los datos. Cada nodo interno representa una prueba sobre un atributo, cada rama representa el resultado de la prueba y cada hoja representa una clase o resultado final, permitiendo clasificar o predecir resultados a partir de nuevas entradas."
}


In [7]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.5,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura jerárquica para tomar decisiones basadas en características de los datos. Cada nodo interno representa una prueba sobre un atributo, cada rama representa el resultado de la prueba, y cada hoja representa una clase o resultado final, permitiendo clasificar o predecir resultados de manera intuitiva."
}


In [8]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.9,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "promt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura en forma de árbol para tomar decisiones basadas en características de entrada. Cada nodo interno representa una prueba sobre una característica, cada rama representa el resultado de esa prueba, y cada hoja representa una decisión final o clasificación."
}


Al ir aumentando el temperautre, la salida se va volviendo creativa